In [9]:
import logging
import os
import sys

# Add working dir to path
sys.path.insert(0,os.environ['PWD'])

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

LOG = logging.getLogger(__name__)

In [10]:
import itertools
import hashlib
import logging
import os
import ssl
import time


import aiohttp
import asyncio

from contextlib import asynccontextmanager

from krill_api import KrillApiClient

In [11]:
TOKEN = os.environ['KRILL_TOKEN']
KRILL_URL = os.environ['KRILL_URL']
KRILL_CERT_PATH = os.environ['KRILL_CERT_PATH']

# Dump the certificate in DER format like
# > openssl s_client -showcerts \
#     -servername krill.host \
#     -connect krill.host:3000 2>/dev/null | openssl x509 -inform pem -outform DER -out {KRILL_CERT_PATH}.der
#


with open(KRILL_CERT_PATH, 'rb') as f:
    digest = hashlib.sha256(f.read()).digest()
    
logging.basicConfig()
LOG = logging.getLogger(__name__)

In [12]:
async with aiohttp.ClientSession(headers={'Authorization': f"Bearer {TOKEN}", "Accept": "application/json"}) as session:
    async with session.get(KRILL_URL + "/stats/info", ssl=aiohttp.Fingerprint(digest)) as r:
        json_body = await r.json()
        print(json_body)

{'version': '0.12.0', 'started': 1669969880}


In [13]:
async with KrillApiClient(KRILL_URL, TOKEN, aiohttp.Fingerprint(digest)) as client:
    print(await client.info())


KrillStatusInfo(version='0.12.0', started=1669969880)


In [18]:
"""
Get the status of all krill CAs
"""
async with KrillApiClient(KRILL_URL, TOKEN, aiohttp.Fingerprint(digest)) as client:
    t0 = time.time()

    res = await client.list()
    
    t1 = time.time()
    
    status_lists = await asyncio.gather(*[
        client.ca_parent_status(r.handle)
        for r in res
    ])
    
    t2 = time.time()
    statuses = list(itertools.chain.from_iterable(status_lists))
    
LOG.info(f"Loaded a list of %d CAs in %.3fs. Loaded %d CA details in %.3fs", len(res), t1-t0, len(status_lists), t2-t1)

INFO:__main__:Loaded a list of 1281 CAs in 1.878s. Loaded 1281 CA details in 8.492s


In [17]:
"""
Delete CAs with failing parent communication
"""
async with KrillApiClient(KRILL_URL, TOKEN, aiohttp.Fingerprint(digest)) as client:
    for s in statuses:
        if s.last_exchange.result != 'Success':
            LOG.info("Deleting CA {} because of status {}", s.handle, s.last_exchange)
            LOG.info("result: {}", await client.ca_delete(s.handle))